In [1]:
import pandas as pd 
import numpy as np
import calendar
from datetime import datetime, date, timedelta

In [2]:
lag_size = [1]
n_lags = [7]
lags = []
lag = 0
for i in range(len(lag_size)) :
  n = n_lags[i]
  for j in range(n) :
    lag += lag_size[i]
    lags.append(lag)

fwd = 10

In [26]:
import ccxt
exchange_class = getattr(ccxt, 'binance')
binance  = exchange_class()


def min_ohlcv(dt, pair, limit):
    # UTC native object
    since = calendar.timegm(dt.utctimetuple())*1000
    ohlcv1 = binance.fetch_ohlcv(symbol=pair, timeframe='1m', since=since, limit=limit)
    ohlcv2 = binance.fetch_ohlcv(symbol=pair, timeframe='1m', since=since, limit=limit)
    ohlcv = ohlcv1 + ohlcv2
    return ohlcv

def ohlcv(dt, pair, period='1d'):
    ohlcv = []
    limit = 1000
    if period == '1m':
        limit = 720
    elif period == '1d':
        limit = 365
    elif period == '1h':
        limit = 24
    elif period == '5m':
        limit = 288
    for i in dt:
        start_dt = datetime.strptime(i, "%Y%m%d")
        since = calendar.timegm(start_dt.utctimetuple())*1000
        if period == '1m':
            ohlcv.extend(min_ohlcv(start_dt, pair, limit))
        else:
            ohlcv.extend(binance.fetch_ohlcv(symbol=pair, timeframe=period, since=since, limit=limit))
    df = pd.DataFrame(ohlcv, columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
    df['Time'] = [datetime.fromtimestamp(float(time)/1000) for time in df['Time']]
    df['Open'] = df['Open'].astype(np.float64)
    df['High'] = df['High'].astype(np.float64)
    df['Low'] = df['Low'].astype(np.float64)
    df['Close'] = df['Close'].astype(np.float64)
    df['Volume'] = df['Volume'].astype(np.float64)
    df.set_index('Time', inplace=True)
    return df

from datetime import datetime, date, timedelta

start_day = "20200101" #"20201201"
start_dt = datetime.strptime(start_day, "%Y%m%d")
end_day = "20211104"
end_dt = datetime.strptime(end_day, "%Y%m%d")
days_num = (end_dt - start_dt).days + 1
datelist = [start_dt + timedelta(days=x) for x in range(days_num)]
datelist = [date.strftime("%Y%m%d") for date in datelist]

#fwd = 10
coins = ['BTC' , 'ETH', 'FTM']
df_list = []
for coin in coins : 
    pair = coin + "/USDT"
    df = ohlcv(datelist, pair, '1d')
    df = (df.reset_index()
        .drop_duplicates(subset='Time', keep='first')
        .set_index('Time').sort_index())

    for f in range(fwd) : 
        df_fwd = df.shift(-1*(f+1))
        df['Fwd_Close_' + str(f+1) ] = df_fwd['Close'] / df['Open']
        df['Close_' + str(f+1) ] = df_fwd['Close']
        df['Fwd_High_' + str(f+1) ] = df_fwd['High'] / df['Open']
        df['Fwd_Low_' + str(f+1) ] = df_fwd['Low'] / df['Open']

        # get max & min of high / low 
        highList = []
        lowList = []
        for j in range(f) : 
            highList.append('Fwd_High_' + str(j+1))
            lowList.append('Fwd_Low_' + str(j+1))
        df['Fwd_Max_'  + str(f+1) ] = df[highList].max(axis=1)
        df['Fwd_Min_'  + str(f+1) ] = df[lowList].min(axis=1) 


        #df['High_per'] = df.apply(lambda row : row.High / row.Open, axis = 1)
        #df['Close_per'] = df.apply(lambda row : row.Close / row.Open, axis = 1)
        #df['Low_per'] = df.apply(lambda row : row.Low / row.Open, axis = 1)
    for col in df.columns : 
        if col != "Time" : 
          df = df.rename(columns={col : col + "_" + coin})
    #df.to_csv(path + coin + "_historical.csv")


    df_list.append(df)




In [30]:
for i in range(len(coins)) : 
    df = df_list[i]
    coin = coins[i]
    df = (df.reset_index()
        .drop_duplicates(subset='Time', keep='first')
        .set_index('Time').sort_index())
    df.to_csv("prices_all_1h" + coin + ".csv")

In [31]:
df_fwd[350:400]

,Open,High,Low,Close,Volume,Fwd_Close_1,Close_1,Fwd_High_1,Fwd_Low_1,Fwd_Max_1,...,Fwd_High_8,Fwd_Low_8,Fwd_Max_8,Fwd_Min_8,Fwd_Close_9,Close_9,Fwd_High_9,Fwd_Low_9,Fwd_Max_9,Fwd_Min_9
Time,,,,,,,,,,,,,,,,,,,,,
2020-12-16,0.01730,0.01731,0.01609,0.01663,7.952471e+07,0.961850,0.01664,1.024855,0.932948,NaN,...,1.132948,0.926012,1.040462,0.894220,1.183237,0.02047,1.203468,0.954335,1.132948,0.894220
2020-12-17,0.01664,0.01773,0.01614,0.01664,1.133225e+08,1.045072,0.01739,1.081731,0.979567,NaN,...,1.251202,0.992188,1.177885,0.929688,1.239784,0.02063,1.288462,1.121995,1.251202,0.929688
2020-12-18,0.01665,0.01800,0.01630,0.01739,1.062085e+08,0.980781,0.01633,1.064264,0.932733,NaN,...,1.287688,1.121321,1.250450,0.929129,1.466667,0.02442,1.471471,1.180180,1.287688,0.929129
2020-12-19,0.01737,0.01772,0.01553,0.01633,1.356418e+08,0.955095,0.01659,0.995970,0.890616,NaN,...,1.410478,1.131261,1.234312,0.890616,1.371906,0.02383,1.588946,1.329879,1.410478,0.890616
2020-12-20,0.01634,0.01730,0.01547,0.01659,1.274225e+08,1.034272,0.01690,1.047124,0.974908,NaN,...,1.689106,1.413709,1.499388,0.974908,1.590575,0.02599,1.689106,1.312729,1.689106,0.974908
2020-12-21,0.01658,0.01711,0.01593,0.01690,9.109811e+07,1.036791,0.01719,1.079614,1.000000,NaN,...,1.664656,1.293727,1.664656,0.966224,1.632087,0.02706,1.749095,1.507841,1.664656,0.966224
2020-12-22,0.01690,0.01790,0.01658,0.01719,1.105613e+08,0.969822,0.01639,1.018935,0.949704,NaN,...,1.715976,1.479290,1.633136,0.947929,1.489941,0.02518,1.721302,1.374556,1.715976,0.947929
2020-12-23,0.01719,0.01722,0.01605,0.01639,9.365291e+07,1.088424,0.01871,1.140198,0.931937,NaN,...,1.692263,1.351367,1.687027,0.931937,1.286795,0.02212,1.466550,1.084351,1.692263,0.931937
2020-12-24,0.01641,0.01960,0.01602,0.01871,1.842304e+08,1.247410,0.02047,1.268739,1.006094,NaN,...,1.536258,1.135893,1.772700,1.006094,1.334552,0.02190,1.502742,1.296770,1.772700,1.006094
